#### Importing all the necessary files and modules

In [2]:
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np 

from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dropout,Flatten,Dense, Rescaling
from tensorflow.keras.utils import image_dataset_from_directory


#### Defining constants throughout the notebook

In [3]:
batch_size = 32
height = 150
width = 150
trainPath = '/Users/bhavesh/Documents/GitHub/cognition-3.0/Final-Project/archive/seg_train'
testPath = '/Users/bhavesh/Documents/GitHub/cognition-3.0/Final-Project/archive/seg_test'

#### The Data Function 
This data function will be loading and preprocessing the data 
Preprocessing on the grounds of image size and batch_size

And creating DataSets for model training 


In [4]:
def data(path,labels):
    dataset = image_dataset_from_directory(
        directory = path,
        labels = labels,
        seed = 123,
        image_size=(height,width),
        batch_size = batch_size
    )
    return dataset

trainDS = data(trainPath, labels='inferred')
testDS = data(testPath, labels='inferred')

className = trainDS.class_names
print(className) #analysis on the category 

Found 14034 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.
['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


#### Normalizing data 

In [5]:
standard = Rescaling(1./255)  #normalizing 

trainDS = trainDS.map(lambda x,y: (standard(x), y))
testDS = testDS.map(lambda x,y: (standard(x), y))

### Model Building and Training 

In [6]:
model = Sequential([
     tf.keras.Input(shape=(150,150,3)),
     Conv2D(16,(3,3), activation='relu'),
     MaxPooling2D(2,2),
     Conv2D(32,(3,3), activation='relu'),
     MaxPooling2D(2,2),
     Conv2D(64,(3,3), activation='relu'),
     MaxPooling2D(2,2),
     Flatten(),
     Dense(128, activation='relu'),
     Dense(len(className),activation='softmax')

])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
epochs = 10
history = model.fit(trainDS, validation_data=testDS, epochs=epochs)

Epoch 1/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 68s 153ms/step - accuracy: 0.5603 - loss: 1.1237 - val_accuracy: 0.7417 - val_loss: 0.7191
Epoch 2/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 46s 105ms/step - accuracy: 0.7587 - loss: 0.6628 - val_accuracy: 0.7847 - val_loss: 0.5878
Epoch 3/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 48s 110ms/step - accuracy: 0.8102 - loss: 0.5217 - val_accuracy: 0.8090 - val_loss: 0.5594
Epoch 4/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 46s 106ms/step - accuracy: 0.8695 - loss: 0.3840 - val_accuracy: 0.8170 - val_loss: 0.5739
Epoch 5/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 47s 106ms/step - accuracy: 0.9003 - loss: 0.2798 - val_accuracy: 0.7940 - val_loss: 0.6850
Epoch 6/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 49s 111ms/step - accuracy: 0.9359 - loss: 0.1951 - val_accuracy: 0.8003 - val_loss: 0.7305
Epoch 7/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 52s 118ms/step - accuracy: 0.9603 - loss: 0.1252 - val_accuracy: 0.7950 - val_loss: 0.8638
Epoch 8/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 48s 108ms/step - accuracy: 0.9665 - loss: 0

### Saving Model 

In [13]:
model.save('ultra.keras') # type: ignore
print("Model saved to ultra.keras")

Model saved to ultra.keras


#### A prediction Script is below to predict any img 

In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

model = load_model('ultra.keras')
height = 150
width = 150

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(height, width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  
    img_array = img_array / 255.0  
    return img_array

def predict_image_class(img_path):
    img_array = preprocess_image(img_path)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    return predicted_class[0]

image_path = '/Users/bhavesh/Documents/GitHub/cognition-3.0/Final-Project/archive/seg_pred/4357.jpg'


predicted_class = predict_image_class(image_path)

class_names = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
print(f"The predicted class is: {class_names[predicted_class]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
The predicted class is: sea
